## 1. Import packages

In [189]:
import pickle, os
import jieba  
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans 
from collections import defaultdict
from matplotlib import pyplot as plt
from sklearn.preprocessing import Normalizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity

## 2. Load data

In [136]:
news = pd.read_csv('sqlResult.csv', encoding='gb18030')

print('='*40,"shape of the news dataset",)
print(news.shape)
print('='*40,"information of the news dataset")
print(news.info())
print('='*40,"check the first 5 items of the news")
print(news.head(5))


with open('chinese_stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = [line[:-1] for line in file.readlines()]

print('='*40,"stop words")
print(stopwords[:30])

======================================== shape of the news dataset
(89611, 7)
======================================== information of the news dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 7 columns):
id         89611 non-null int64
author     79396 non-null object
source     89609 non-null object
content    87054 non-null object
feature    89611 non-null object
title      89577 non-null object
url        87144 non-null object
dtypes: int64(1), object(6)
memory usage: 4.8+ MB
None
======================================== check the first 5 items of the news
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余

## 3. Clean data

In [137]:
news = news.dropna(subset=['content'])  # news.dropna(subset=['content'], inplace = True) 
print(news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87054 entries, 0 to 89610
Data columns (total 7 columns):
id         87054 non-null int64
author     76925 non-null object
source     87052 non-null object
content    87054 non-null object
feature    87054 non-null object
title      87054 non-null object
url        87054 non-null object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB
None


In [138]:
# news = news.dropna(subset=['source'])  
# print(news.info())

## 4. Split text  

In [139]:
def split_text(text):
    text = text.replace(' ', '').replace('\n', '').replace('\r', '')
    text2 = jieba.cut(text.strip()) 
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

**print the content of the first item of the news dataset  
print the content of the first item after splited of the dataset**

In [140]:
temp = news.iloc[0].content
print(temp)
print(split_text(temp))

此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。
MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。
当然，关于MIUI 9的确切信息，我们还是等待官方消息。

此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 MIUI8 去年 发布 距今已有 一年 有余 更新换代 当然 MIUI9 确切 信息 等待 官方消息


**split all the text and save it into corpus.pkl**

In [141]:
if os.path.exists('corpus.pkl'):
    with open('corpus.pkl', 'rb') as file:
        corpus = pickle.load(file)
else:
    corpus = news['content'].map(split_text)
    with open('corpus.pkl', 'wb') as file:
        pickle.dump(corpus, file)
        
print(corpus[0])

 当然 MIUI9 确切 信息 等待 官方消息


## 5.  Count word frequency

In [142]:
countVectorizer = CountVectorizer(encoding='gb18030', min_df=0.015)
corpus_vectorizer = countVectorizer.fit_transform(corpus)

tfidfTransformer = TfidfTransformer()
tfidf = tfidfTransformer.fit_transform(corpus_vectorizer)

label = list(map(lambda source:1 if '新华社' in str(source) else 0, news.source))

In [143]:
print(type(corpus_vectorizer))
print(type(tfidf))

print(tfidf.shape)
print('='*40,"the first 3 rows of sparse matrix of tfidf")
print(tfidf[:3])

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
(87054, 884)


## 6. MultinomialNB
### 6.1 Split dataset

In [149]:
x_train, x_test, y_train, y_test = train_test_split(tfidf.toarray(), label, test_size=0.2)

print(x_train.shape)
print(x_test.shape)
print(len(y_train))
print(len(y_test))

(69643, 884)
(17411, 884)
69643
17411


### 6.2 Fit model 

In [162]:
model = MultinomialNB()
model.fit(x_train, y_train)

prediction =  model.predict(tfidf.toarray())
labels = np.array(label)
compare_news_index = pd.DataFrame({'prediction':prediction, 'labels':labels})
print('='*40,"information of compare news")
print(compare_news_index.info())
print(compare_news_index.head(7))

copy_news_index = compare_news_index[(compare_news_index['prediction']==1) & (compare_news_index['labels']==0)]
print('='*40,"information of copy news")
print(copy_news_index.info())
print(copy_news_index.head(7))

xinhuashe_news_index = compare_news_index[(compare_news_index['labels']==1)].index
print('='*40,"information of xinhuashe news")
print(len(xinhuashe_news_index))


======================================== information of compare news
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87054 entries, 0 to 87053
Data columns (total 2 columns):
prediction    87054 non-null int32
labels        87054 non-null int32
dtypes: int32(2)
memory usage: 680.2 KB
None
   prediction  labels
0           0       0
1           0       0
2           0       0
3           1       1
4           1       0
5           0       0
6           0       0
======================================== information of copy news
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2817 entries, 4 to 8555
Data columns (total 2 columns):
prediction    2817 non-null int32
labels        2817 non-null int32
dtypes: int32(2)
memory usage: 44.0 KB
None
    prediction  labels
4            1       0
24           1       0
25           1       0
28           1       0
30           1       0
35           1       0
37           1       0
======================================== information of xinhuash

## 7. Kmeans cluster
### 7.1 Data normalization

In [166]:
normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfidf.toarray())

### 7.2 KMeans model fit

In [184]:
%%time
kmeans = KMeans(n_clusters=25)
k_labels = kmeans.fit_predict(scaled_array)

len(k_labels)

Wall time: 4min 41s


87054

In [185]:
def find_similar_text(cpindex, top=10):
    dist_dict = {i:cosine_similarity(tfidf[cpindex], tfidf[i]) for i in class_id[id_class[cpindex]]}
    return sorted(dist_dict.items(), key=lambda x:x[1][0], reverse=True)[:top]

In [186]:
%%time
id_class = {index:class_ for index, class_ in enumerate(k_labels)}
class_id = defaultdict(set)
for index, class_ in id_class.items():
    if index in xinhuashe_news_index.to_list():
        class_id[class_].add(index)

Wall time: 2min 53s


In [194]:
%%time
cpindex = 3352
similar_list = find_similar_text(cpindex)
similar2 = similar_list[0][0]

print('='*40,"相似距离：\n")
print(similar_list)

print('='*40,"怀疑抄袭：\n")
print(news.iloc[cpindex].content)

print('='*40,"相似原文:\n")
print(news.iloc[similar2].content)  

======================================== 相似距离：

[(3134, array([[0.96849134]])), (63511, array([[0.94643198]])), (29441, array([[0.94283416]])), (3218, array([[0.87621892]])), (29615, array([[0.86936328]])), (29888, array([[0.86215862]])), (64046, array([[0.85278235]])), (29777, array([[0.84875422]])), (64758, array([[0.73394798]])), (29973, array([[0.7252432]]))]
======================================== 怀疑抄袭：

　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落